# Lab 2. Solar Panel Instructions

## はじめに

このノートブックでは、Amazon Bedrock Agents で 2 番目のサブエージェントを作成する方法を説明します。

このエージェントには、ソーラーパネルのインストール方法とメンテナンス方法に関する手順が含まれており、顧客はエージェントに [Amazon Bedrock ナレッジベース](https://aws.amazon.com/bedrock/knowledge-bases/) からこれらの情報を返すように依頼できます。

組織は、基盤モデル (FM) に最新の独自の情報を装備するために、Retrieval Augmented Generation (RAG) を使用します。これは、会社のデータソースからデータを取得し、プロンプトを充実させて、より関連性が高く正確な応答を提供する手法です。

Amazon Bedrock Knowledge Bases は、Bedrock の完全マネージド機能であり、データソースへのカスタム統合を構築したり、データフローを管理したりすることなく、取り込みから取得、プロンプトの拡張までの RAG ワークフロー全体を実装するのに役立ちます。

エージェントのコンテキストでは、質問に対する回答がナレッジ ベースに存在しない場合、顧客はエージェントにサポート チケットの作成を依頼して、人間によるサポートを受け、質問に回答してもらうことができます。

以下は、このモジュール上に構築されるアーキテクチャの一部を表しています。

![アーキテクチャ](img/solar_panel_agent.png)

## Setup

boto3 のバージョンが最新であることを確認してください

そうでない場合は、[notebook 1](../1-energy-forecast/1_forecasting_agent.ipynb) を返さず、セットアップ ブロックを再度実行してください。

In [ ]:
!pip freeze | grep boto3

## エージェントの作成

このセクションでは、ノートブック全体でヘルパーとして機能するグローバル変数を宣言し、2 番目のエージェントの作成を開始します。

In [ ]:
import boto3
import os

sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name
account_id_suffix = account_id[:3]
agent_suffix = f"{region}-{account_id_suffix}"
s3_client = boto3.client('s3', region)
bedrock_client = boto3.client('bedrock-runtime', region)

agent_foundation_model = [
    'anthropic.claude-3-haiku-20240307-v1:0',
    'anthropic.claude-3-sonnet-20240229-v1:0',
    'anthropic.claude-3-5-sonnet-20240620-v1:0'
]

In [ ]:
solar_agent_name = f"solar-p-{agent_suffix}"

solar_lambda_name = f"fn-solar-p-{agent_suffix}"

solar_agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{solar_agent_name}'

dynamodb_table = f"{solar_agent_name}-table"
dynamodb_pk = "customer_id"
dynamodb_sk = "ticket_id"

dynamoDB_args = [dynamodb_table, dynamodb_pk, dynamodb_sk]

knowledge_base_name = f'{solar_agent_name}-kb'
suffix = f"{region}-{account_id}"

knowledge_base_description = "KB containing solar panel instructions for installation and maintenance"
bucket_name = f'solar-p-kb-{suffix}'

### ヘルパー関数のインポート

次のセクションでは、`bedrock_agent_helper.py` と `knowledge_base_helper` を Python パスに追加して、ファイルが認識され、その機能を呼び出すことができるようにします。

次に、ヘルパー クラス `bedrock_agent_helper.py` と `knowledge_base_helper.py` をインポートします。

これらのファイルには、ラボをスムーズに実行することに重点を置いたヘルパー クラスが含まれています。

Bedrock とのすべてのやり取りは、これらのクラスによって処理されます。

このラボで呼び出すメソッドは次のとおりです:

`agents.py` の場合:
- `create_agent`: 新しいエージェントとそれぞれの IAM ロールを作成します
- `add_action_group_with_lambda`: Lambda 関数を作成し、以前に作成したエージェントのアクション グループとして追加します
- `create_agent_alias`: このエージェントのエイリアスを作成します
- `invoke`: エージェントを実行します

`knowledge_bases.py` の場合:
- `create_or_retrieve_knowledge_base`: Amazon Bedrock にナレッジ ベースが存在しない場合は作成するか、以前に作成したナレッジ ベースに関する情報を取得します。
- `synchronize_data`: S3 上のファイルを読み取り、テキスト情報をベクトルに変換して、その情報を Vector Database に追加します。

In [ ]:
import sys

sys.path.insert(0, ".")
sys.path.insert(1, "..")

from utils.bedrock_agent_helper import (
    AgentsForAmazonBedrock
)
from utils.knowledge_base_helper import (
    KnowledgeBasesForAmazonBedrock
)
agents = AgentsForAmazonBedrock()
kb = KnowledgeBasesForAmazonBedrock()

## ナレッジベースを作成してロードする

このセクションでは、Amazon Bedrock ナレッジベースを作成し、そこにデータを取り込みます。

次の手順では、ナレッジベースに取り込むために使用するデータを生成します。このデータは、ソーラーパネルの取り扱い方法に関する手順で構成されます

**この作成プロセスには数分かかります。**

In [ ]:
%%time
kb_id, ds_id = kb.create_or_retrieve_knowledge_base(
    knowledge_base_name,
    knowledge_base_description,
    bucket_name
)

print(f"Knowledge Base ID: {kb_id}")
print(f"Data Source ID: {ds_id}")

### S3 にロードする合成データを作成する

他の場所でデータを取得する代わりに、Amazon Bedrock の LLM を使用してデータを生成します。
生成されるこの偽のデータは、S3 バケットにアップロードされ、Amazon Bedrock ナレッジベースに追加されます。

In [ ]:
path = "kb_documents"
# Check whether the specified path exists or not
isExist = os.path.exists(path)
if not isExist:
   # Create a n ew directory if it does not exist
   os.makedirs(path)
   print("The {} directory was created!".format(path))
else:
   print("The {} directory already exists!".format(path))

Bedrock 上で LLM を呼び出し、Python を使用してローカル ファイルを書き込むヘルパー メソッドを作成する

In [ ]:
def invoke_bedrock_generate_energy_files(prompt):
    message_list = []

    initial_message = {
        "role": "user",
        "content": [
            { "text": prompt } 
        ],
    }

    message_list.append(initial_message)

    response = bedrock_client.converse(
        modelId=agent_foundation_model[0],
        messages=message_list,
        inferenceConfig={
            "maxTokens": 2048,
            "temperature": 0
        },
    )

    return response['output']['message']


def write_file(file_name, content):
    f = open(file_name, 'w')
    f.write(content)
    f.close

ソーラーパネルの設置方法を説明したファイルを 1 つ生成します。

In [ ]:
text_generation_energy_instructions = '''
    You will be act as an expert on clean energy.
    You will generate a step-by-step on how to install a solar panel at home.
    You know the following fictional solar panel models: Sunpower X, Sunpower Y
    and Sunpower double-X. For each one of those models, provide some general
    model description and its features. Next provide a numbered list describing
    how to install each model of solar panel. Include information about how to
    ensure compliance with energy rules.

    Answer only with the instructions and solar panel descriptions.
    Avoid answer with afirmations like: "OK, I can generate it,",
    "As an expert on clean energy, I ", or "Yes, please find following example."
    Be direct and only reply the instructions and descriptions.
'''

solar_energy_file_name = 'solar-panel-instructions.txt'

response_message = invoke_bedrock_generate_energy_files(
    text_generation_energy_instructions
)
description_and_instructions = response_message['content'][0]['text']
print(f"Generated data:\n{description_and_instructions}")
write_file(
    '{}/{}'.format(path, solar_energy_file_name),
    description_and_instructions
)

### データプロンプトを生成しています
ソーラーパネルのメンテナンス方法を説明する別のファイルを生成しています。

In [ ]:
text_generation_energy_instructions = f'''
    You will be act as an expert on clean energy.
    You know the following fictional solar panel models: Sunpower X, Sunpower Y
    and Sunpower double-X. Here is are some descriptions of the different
    models and how to install them:
    <description_and_instructions>
    {description_and_instructions}
    </description_and_instructions>
    Generate a step-by-step instructions on how to do maintenance on each of
    those models at a regular home. Include information about how to
    ensure consistent compliance with energy rules.
    Just answer in a numbered list.
'''

solar_energy_file_name = 'solar-panel-maintenance.txt'

response_message = invoke_bedrock_generate_energy_files(
    text_generation_energy_instructions
)
print(f"Generated data:\n{response_message['content'][0]['text']}")

write_file(
    '{}/{}'.format(path, solar_energy_file_name),
    response_message['content'][0]['text']
)


### S3 へのデータのアップロード
生成されたファイルを S3 バケットにアップロードします。

In [ ]:
def upload_directory(path, bucket_name):
    for root,dirs,files in os.walk(path):
        for file in files:
            file_to_upload = os.path.join(root,file)
            print(f"uploading file {file_to_upload} to {bucket_name}")
            s3_client.upload_file(file_to_upload,bucket_name,file)

### ナレッジベースの同期
データが s3 バケットで利用できるようになったので、ナレッジベースに同期してみましょう。

In [ ]:
upload_directory("kb_documents", bucket_name)

# sync knowledge base
kb.synchronize_data(kb_id, ds_id)

## エージェントの作成

ナレッジ ベースと、例外ワークフロー (KB に情報が見つからない場合) を処理するための Lambda アクション グループを持つソーラー パネル エージェントを作成します。

このエージェントでは、次の手順を使用します:
```
あなたは、ソーラー パネルの設置とメンテナンスのガイダンスで顧客を支援するソーラー エネルギー アシスタントです。

次のような能力があります:
1. 設置手順の提供
2. メンテナンス手順の提供
3. 一般的な問題のトラブルシューティング
4. 専門家による支援のためのサポート チケットの作成

コア ビヘイビア:
1. 顧客に詳細を尋ねる前に、常に利用可能な情報を使用する
2. プロフェッショナルでありながら親しみやすい口調を維持する
3. 明確で直接的な回答を提供する
4. 技術情報をわかりやすい方法で提示する
5. ナレッジ ベースにない情報を決して作成しない

サポート チケット プロトコル:
- 専門家レベルの問題に対してのみチケットを生成する
- チケットを作成するときは、ケース ID のみを使用して応答する
- 自分の範囲を超える専門家のアドバイスは提供しない

応答スタイル:
- 役立つ、ソリューション指向であること
- 明確で実用的な言葉を使用する
- 実用的なガイダンスに焦点を当てる
- 自然な会話の流れを維持する
- 簡潔でありながら有益な情報を提供する
- ユーザーが必要としない余分な情報を追加しない
```
また、ナレッジ ベースをソーラー パネルに関する情報に接続します

```
顧客がソーラー パネルの設置とメンテナンスについて質問したときに、ナレッジ ベースにアクセスします
```
そして、エージェントが次のツールを利用できるようになります:
- `open_ticket`: 新しいサポート チケットを開く
- `get_ticket_status`: 既存のチケットの現在のステータスを取得する

In [ ]:
kb_info = kb.get_kb(kb_id)
kb_arn = kb_info['knowledgeBase']['knowledgeBaseArn']

In [ ]:
kb_config = {
    'kb_id': kb_id,
    'kb_instruction': 'Access the knowledge base when customers ask about to install and maintain solar panels'
}

In [ ]:
agent_instruction = """You are a Solar Energy Assistant that helps customers with solar panel installation and maintenance guidance.

Your capabilities include:
1. Providing installation instructions
2. Offering maintenance procedures
3. Troubleshooting common issues
4. Creating support tickets for specialist assistance

Core behaviors:
1. Always use available information before asking customers for additional details
2. Maintain a professional yet approachable tone
3. Provide clear, direct answers
4. Present technical information in an easy-to-understand manner

Support ticket protocol:
- Only generate tickets for specialist-level issues
- Respond exclusively with case ID when creating tickets
- Decline providing specialist advice beyond your scope

Response style:
- Be helpful and solution-oriented
- Use clear, practical language
- Focus on actionable guidance
- Maintain natural conversation flow
- Be concise yet informative
- Do not add extra information not required by the user"""

agent_description = """You are a solar energy helper bot. 
    You can retrieve information on how to install and do maintenance on solar panels"""

solar_agent = agents.create_agent(
    solar_agent_name,
    agent_description,
    agent_instruction,
    agent_foundation_model,
    kb_arns=[kb_arn]
)
solar_agent

In [ ]:
agents.associate_kb_with_agent(
    solar_agent[0], kb_config['kb_instruction'], kb_config['kb_id']
)

### アクション グループの作成

このセッションでは、サポート チケットを処理するアクション グループを作成し、それをエージェントに関連付けます。そのためには、まずエージェントのアクションを実行する Lambda 関数コードを作成します。次に、関数の詳細を使用してエージェントが実行できるアクションを定義します。前のエージェントと同様に、OpenAPI スキーマを使用して利用可能なアクションを定義することもできます。

#### Lambda 関数の作成
まず、Lambda 関数を作成しましょう

In [ ]:
%%writefile solar_energy.py
import os
import json
import uuid
import boto3

from boto3.dynamodb.conditions import Key, Attr

dynamodb_resource = boto3.resource('dynamodb')
dynamodb_table = os.getenv('dynamodb_table')
dynamodb_pk = os.getenv('dynamodb_pk')
dynamodb_sk = os.getenv('dynamodb_sk')

def get_named_parameter(event, name):
    try:
        return next(item for item in event['parameters'] if item['name'] == name)['value']
    except:
        return None
    
def populate_function_response(event, response_body):
    return {'response': {'actionGroup': event['actionGroup'], 'function': event['function'],
                'functionResponse': {'responseBody': {'TEXT': {'body': str(response_body)}}}}}

def put_dynamodb(table_name, item):
    table = dynamodb_resource.Table(table_name)
    resp = table.put_item(Item=item)
    return resp

def read_dynamodb(table_name: str, 
                   pk_field: str,
                   pk_value: str,
                   sk_field: str=None, 
                   sk_value: str=None):
    try:
        table = dynamodb_resource.Table(table_name)
        # Create expression
        if sk_value:
            key_expression = Key(pk_field).eq(pk_value) & Key(sk_field).begins_with(sk_value)
        else:
            key_expression = Key(pk_field).eq(pk_value)

        query_data = table.query(
            KeyConditionExpression=key_expression
        )
        
        return query_data['Items']
    except Exception:
        print(f'Error querying table: {table_name}.')

def open_ticket(customer_id, msg):
    ticket_id = str(uuid.uuid1())
    item = {
        'ticket_id': ticket_id,
        'customer_id': customer_id,
        'description': msg,
        'status': 'created'
    }
    resp = put_dynamodb(dynamodb_table, item)
    print(resp)
    return "Thanks for contact customer {}! Your support case was generated with ID: {}".format(
        customer_id, ticket_id
    )

def get_ticket_status(customer_id,
                      ticket_id: str=None):
    return read_dynamodb(dynamodb_table, 
                         dynamodb_pk,
                         customer_id,
                         dynamodb_sk,
                         ticket_id)

def lambda_handler(event, context):
    print(event)
    
    # name of the function that should be invoked
    function = event.get('function', '')

    # parameters to invoke function with
    parameters = event.get('parameters', [])
    customer_id = get_named_parameter(event, "customer_id")

    if function == 'open_ticket':
        msg = get_named_parameter(event, "msg")
        result = open_ticket(customer_id, msg)
    elif function == 'get_ticket_status':
        ticket_id = get_named_parameter(event, "ticket_id")
        result = get_ticket_status(customer_id, ticket_id)
    else:
        result = f"Error, function '{function}' not recognized"

    response = populate_function_response(event, result)
    print(response)
    return response

### 利用可能なアクションの定義
エージェントが実行できるアクションを定義します

In [ ]:
functions_def =[
    {
        "name": "open_ticket",
        "description": """Create a ticket to get help with information related with solar panel or clean energy""",
        "parameters": {
            "customer_id": {
                "description": "Unique customer identifier",
                "required": True,
                "type": "string"
            },
            "msg": {
                "description": "The reason why customer is opening a ticket",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        "name": "get_ticket_status",
        "description": """get the status of an existing ticket""",
        "parameters": {
            "customer_id": {
                "description": "Unique customer identifier",
                "required": True,
                "type": "string"
            },
            "ticket_id": {
                "description": "Unique ticket identifier",
                "required": False,
                "type": "string"
            }
        }
    }
]

#### アクション グループをエージェントに関連付ける
最後に、新しいアクション グループを以前に作成したエージェントに関連付けることができます

In [ ]:
resp = agents.add_action_group_with_lambda(
    agent_name=solar_agent_name,
    lambda_function_name=solar_lambda_name,
    source_code_file="solar_energy.py",
    agent_functions=functions_def,
    agent_action_group_name="solar_energy_actions",
    agent_action_group_description="Function to open an energy ticket for a user or get status from an opened ticket",
    dynamo_args=dynamoDB_args
)

## エージェントのテスト

では、作成したエージェントが機能していることを確認するために、テストをいくつか実行してみましょう。そのためには、エージェントのドラフト バージョンを呼び出すことができるテスト エイリアス `TSTALIASID` を使用します

### メンテナンスに関する質問のテスト
まず、既存のソーラー パネルのメンテナンスに関する質問をしてみましょう

In [ ]:
%%time
response = agents.invoke(
    "how can I check if my Sunpower double-X solar panel eletrical consumption is compliant with energy rules?", 
    solar_agent[0], enable_trace=True
)
print("====================")
print(response)

### テストインストールの質問
次に、新しいソーラーパネルの設置に関する質問をしましょう

In [ ]:
%%time
response = agents.invoke(
    "how can I install my Sunpower Y solar panel at home?", 
    solar_agent[0], enable_trace=True
)
print("====================")
print(response)

### パーソナライズされたサポートのテスト
では、サポート チケットを作成しましょう。そのためには、顧客 ID を渡してサポート チケットの作成プロセスをトリガーします。

In [ ]:
%%time
ticket = agents.invoke(
    "Can I get support to install my Sunpower X solar panel? My customer id is 1", 
    solar_agent[0], enable_trace=True
)
print("====================")
print(ticket)

### サポート チケットの詳細を取得するテスト
では、サポート チケットの詳細を取得しましょう。そのためには、前のクエリで生成されたチケット ID を `create_ticket` に提供する必要があります。

In [ ]:
%%time
response = agents.invoke(
    "Can I get details on my ticket? My customer id is 1 get my ticket id from our previous conversation {}".format(ticket), 
    solar_agent[0], enable_trace=True
)
print("====================")
print(response)

### オープンサポートチケットの取得のテスト
エージェントが複数のオープンケースの詳細を取得できるかどうかも確認しましょう。そのためには、まず新しいサポートケースを作成します。

In [ ]:
%%time
response = agents.invoke(
    "Can I get support to review my Sunpower double-X solar panel consumption? My customer id is 1", 
    solar_agent[0], enable_trace=True
)
print("====================")
print(response)

次に、すべてのオープンチケットの概要をリクエストします

In [ ]:
%%time
response = agents.invoke(
    "Can I get all tickets that I have? My customer id is 1", 
    solar_agent[0], enable_trace=True
)
print("====================")
print(response)

## エイリアスの作成

ご覧のとおり、エージェントを `TSTALIASID` とともに使用してタスクを完了できます。
ただし、マルチエージェント コラボレーションの場合は、最初にエージェントをテストし、完全に機能するようになってからのみ使用することが想定されます。
したがって、マルチエージェント コラボレーションでエージェントをサブエージェントとして使用するには、まずエージェント エイリアスを作成し、それを新しいバージョンに接続する必要があります。

エージェントをテストして検証したので、次にそのエイリアスを作成しましょう。

In [ ]:
solar_agent_alias_id, solar_agent_alias_arn = agents.create_agent_alias(
    solar_agent[0], 'v1'
)
solar_agent_id = solar_agent[0]

次のノートブックで使用する環境変数を保存します。

In [ ]:
solar_agent_arn = agents.get_agent_arn_by_name(solar_agent_name)
solar_kb = knowledge_base_name
solar_dynamodb = dynamodb_table

%store solar_agent_arn
%store solar_agent_alias_arn
%store solar_agent_alias_id
%store solar_lambda_name
%store solar_agent_name
%store solar_agent_id
%store solar_kb
%store solar_dynamodb

In [ ]:
solar_agent_arn, solar_agent_alias_arn, solar_agent_alias_id

## 次のステップ
おめでとうございます! これでソーラーパネルエージェントが作成されました。次はピークローダーマネージャーエージェントを作成します